In [ ]:
# Problem 1 - Camera Callibration

# Importing packages
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Criteria for cornersSubPix() function. Stops after certain iterations,
# epsilon or accuracy is reached
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Determine the pattern size and folder with images
a = 7
b = 7
folder = "test_folder" 


def calibrateCamera(a,b,folder) :

    # "create" object vertices by creating possible corners such as (0,0,0),(1,0,0), ... (5,5,0), ... (a,b,0)
    # third component is 0 since we are keeping the chessboard on a plane and are moving the camera
    possiblePoints3d = np.zeros((a*b,3), np.float32)
    possiblePoints3d[:,:2] = np.mgrid[0:a,0:b].T.reshape(-1,2)

    # Store 3D points (object) and 2D points (image)
    points3d = [] # 3d points in real world space
    points2d = [] # 2d points in image plane.

    found = 0                                                   # count the pictures read 
    images = []                                                 # store images in this array
    for filename in os.listdir(folder):
        path = os.path.join(folder,filename)                    # create path for image you want to read
        img = cv2.imread(path)                                  # read image
        
        # Downscale image because photos shot on iPhone are too big
        p = 0.25                                                # select downscaling factor                         
        new_width = int(img.shape[1] * p)                       # calculate new width
        new_height = int(img.shape[0] * p)                      # calculate new height
        img = cv2.resize(img, (new_width, new_height))          # resize
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)            # convert to gray image
        
        # Can show images for debugging/other purposes
        #cv2.imshow('img',gray)
        #cv2.waitKey(300)
        #cv2.waitKey(3)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (a,b), None)


        # If we found corners/the pattern, add them to points3d and the image points (corners) after refining them
        if ret == True:
            found += 1 
            points3d.append(possiblePoints3d)
            corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria) # add corners after refining
            points2d.append(corners2)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (a,b), corners2, ret)
            cv2.imshow('img', img)
            cv2.waitKey(500)

    cv2.destroyAllWindows()
    # Callibrate the camera with the set of 3D and 2D points and size of gray image
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(points3d, points2d, gray.shape[::-1], None, None)
    print("Number of images used for calibration: ", found)
    return ret, mtx, dist, rvecs, tvecs



def plotCameraLoc(tvecs):
    # Create plot window
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(111, projection='3d')
    
    # Get coordinates from Tvec, obtained from calibrateCamera()
    for arrs in tvecs:
        x_coord = arrs[0]
        y_coord = arrs[1]
        z_coord = arrs[2]
        ax.scatter(x_coord,y_coord,z_coord) # add all coordinates to plot

    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.show()


# Run actual program
ret, mtx, dist, rvecs, tvecs = calibrateCamera(a,b,folder)
plotCameraLoc(tvecs)

print("K matrix = ", mtx)
print("\nR matrices = ", rvecs)
print("\n T vectors = ", tvecs)

